In [1]:
import pandas as pd
import os
import numpy as np
from nltk.corpus import stopwords
import codecs

In [2]:
basepath = 'dataset/aclImdb'

In [3]:
labels = {'pos':1, 'neg':0}

In [4]:
df = pd.DataFrame()

In [5]:
# train = os.listdir(basepath)
# #print(train)
# test = os.listdir(basepath+'/train')
# #print(test)
# neg = os.listdir(basepath+'/train/neg')
# #print(neg)

# try:
#     for i in neg:
#         #df = pd.concat(df, pd.read_csv(i),1)
#         df1 = pd.DataFrame()
#         df1=pd.read_csv(basepath+'/train/neg/'+i, delimiter=' ')
#         #print(df1)
#         df = df.append(df1)
#         df.head()
# except EOFError:
#     pass


    

In [6]:
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath,s,l)
        for fil in os.listdir(path):
            if fil.endswith('.txt'):
                with codecs.open (os.path.join(path,fil),'r', encoding='utf-8', errors='ignore') as f:
                    txt = f.read()
            df = df.append([[txt,labels[l]]], ignore_index=True)

In [7]:
df.columns=['review','sentiment']

In [8]:
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [9]:
df.shape

(50001, 2)

In [10]:
df.sentiment.value_counts()

0    25001
1    25000
Name: sentiment, dtype: int64

In [11]:
from sklearn.utils import shuffle

In [12]:
df = shuffle(df)

In [13]:
df.to_csv('movie_review1.csv', index=True)

In [14]:
#import nltk
#nltk.download('stopwords')

In [15]:
stop = stopwords.words('english')

In [104]:
df_new = pd.read_csv('movie_review1.csv')

In [105]:
df_new.head()

,Unnamed: 0,review,sentiment
0,19489,The operative rule in the making of this film ...,0
1,13745,The concept of this movie is pretty compelling...,0
2,9089,"I was at the world premier of this movie, and ...",1
3,43234,"Isabel Allende's magical, lyrical novel about ...",0
4,28389,This is a complex film that explores the effec...,1


In [106]:
df_new.loc[0, 'review'][-50:]

'e movie seems to revolve around. Seriously - 2/10.'

In [108]:
df_new[ df_new.review.str.contains('<br /><br />', na=False)][:5]

,Unnamed: 0,review,sentiment
0,19489,The operative rule in the making of this film ...,0
2,9089,"I was at the world premier of this movie, and ...",1
4,28389,This is a complex film that explores the effec...,1
5,36856,The Man in the White Suit is one of those deli...,1
8,31253,This is the moving tale of Scotland's legendar...,1


In [109]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',str(text))
    emoticons = re.findall('(?::|;|=)(?:-)',text)
    text = (re.sub('[\W]+',' ',str(text.lower()))+' '.join(emoticons).replace('-',''))
    return text

In [110]:
preprocessor('</a>This is a test :-)!</a>')

'this is a test :'

In [111]:
df_new['review'] = df_new['review'].apply(preprocessor)

In [112]:
df_new.review[:20]

0     the operative rule in the making of this film ...
1     the concept of this movie is pretty compelling...
2     i was at the world premier of this movie and h...
3     isabel allende s magical lyrical novel about t...
4     this is a complex film that explores the effec...
5     the man in the white suit is one of those deli...
6     as is nearly always the case when britain come...
7     bedrooms and hallways gives its audience a loo...
8     this is the moving tale of scotland s legendar...
9     this film starts with a pedestrian setup befor...
10    cheesy 80 s horror co starring genre favs ken ...
11    i was expecting a lot more of this film than w...
12    as a big fan of the original film it s hard to...
13    this is a refreshing enjoyable movie if you en...
14    the movie is a bit thin after reading the book...
15    i just love this show it s so funny and cool k...
16    it is fascinating how this title manages to sl...
17    i loved this movie it is a very simple plo

In [113]:
X_train = df_new.loc[:2500, 'review'].values
y_train = df_new.loc[:2500, 'sentiment'].values
X_test = df_new.loc[2500:5000, 'review'].values
y_test = df_new.loc[2500:5000, 'sentiment'].values

In [114]:
X_train[:1]

array(['the operative rule in the making of this film seems to have been never make a 1 minute scene when you can make it a 10 minute scene this was a principle set right from the start with an interminably long portrayal of the graduation of the harvard class of 1870 the point of that scene i suppose was to introduce some of the primary figures in the story and give a bit of their background which is somewhat effective when comparing the idealism of the harvard graduation ceremony to the realism of life in johnson county wyoming but it just keeps going and going and that sets the stage for a film that features repeated stretches of mind numbing nothingness made even worse by the fact that i found a significant amount of the dialogue to be almost incoherent in the end i couldn t even watch this in one sitting i got through about half of it and had to set it aside for a couple of days before i could drag myself back to see how it turned out my reaction to this movie in many ways is a sh

In [115]:
print (np.bincount(y_test))
print(np.bincount(y_train))

[1238 1263]
[1259 1242]


In [116]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV    #hyper-parameter tunning
from sklearn.model_selection import cross_val_score  #avg testing score

In [117]:
tfidf = TfidfVectorizer(stop_words='english')
param_grid = {'clf__C':[1.0,10.0,100.0]}    # this will map the parameter C of logistic regression

In [118]:
lr_tfidf = Pipeline([('vect',tfidf),('clf', LogisticRegression())])  #vect for tunning parameter of tfids
#if need to aply pca, that should also be put in the pipeline

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy')

In [119]:
gs_lr_tfidf.fit(X_train, y_train)   #need not to be reshaped, it will be reshaped in the pipeline

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': [1.0, 10.0, 100.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [120]:
X_train.shape

(2501,)

In [121]:
#Alternative of pipeline
# X_new = tfidf.fit_transform(X_train)

# log = LogisticRegression()
# log.fit_transform(X_train)


In [122]:
gs_lr_tfidf.best_params_

{'clf__C': 10.0}

In [123]:
gs_lr_tfidf.best_score_

0.8348660535785686

In [124]:
gs_lr_tfidf.best_estimator_.steps

[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [125]:
clf = gs_lr_tfidf.best_estimator_

In [126]:
clf.score(X_test, y_test)

0.846061575369852

In [127]:
from sklearn.naive_bayes import MultinomialNB

# to make same model using multinomial
# 1) using make_pipeline
# from sklearn.pipeline import make_pipeline

# pipe = make_pipeline(CountVectorizer(),MultinomialNB())

# pipe.fit(X_train,y_train)

# pred = pipe.predict(X_test)

# print(pred)

# pipe.score(X_test, y_test)

# pipe.score(X_train, y_train)

In [128]:
nb = Pipeline([('vect', tfidf), ('clf', MultinomialNB())])

In [129]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [130]:
nb.score(X_train, y_train)

0.9652139144342263

In [131]:
nb.score(X_test, y_test)

0.8092762894842063

In [132]:
cv = CountVectorizer()

In [133]:
X_new = cv.fit_transform(X_train)
X_test_n = cv.transform(X_test)

In [134]:
nb1 = MultinomialNB()

In [135]:
nb1.fit(X_new, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [136]:
pred = nb1.predict(X_test_n)

In [140]:
nb1.score(X_test_n, y_test)

0.797281087564974

In [141]:
ord('s')+ord('d')+ord('f')

317